In [49]:
# In this exercise, try to write a program that
# will resample particles according to their weights.
# Particles with higher weights should be sampled
# more frequently (in proportion to their weight).

# Don't modify anything below. Please scroll to the 
# bottom to enter your code.

from math import *
import random

landmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0

class robot:
    def __init__(self):
        self.x = random.random() * world_size
        self.y = random.random() * world_size
        self.orientation = random.random() * 2.0 * pi
        self.forward_noise = 0.0;
        self.turn_noise    = 0.0;
        self.sense_noise   = 0.0;
    
    def set(self, new_x, new_y, new_orientation):
        if new_x < 0 or new_x >= world_size:
            raise (ValueError, 'X coordinate out of bound')
        if new_y < 0 or new_y >= world_size:
            raise (ValueError, 'Y coordinate out of bound')
        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise (ValueError, 'Orientation must be in [0..2pi]')
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)
    
    
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.forward_noise = float(new_f_noise);
        self.turn_noise    = float(new_t_noise);
        self.sense_noise   = float(new_s_noise);
    
    
    def sense(self):
        Z = []
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            dist += random.gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z
    
    
    def move(self, turn, forward):
        if forward < 0:
            raise (ValueError, 'Robot cant move backwards')
        
        # turn, and add randomness to the turning command
        orientation = self.orientation + float(turn) + random.gauss(0.0, self.turn_noise)
        orientation %= 2 * pi
        
        # move, and add randomness to the motion command
        dist = float(forward) + random.gauss(0.0, self.forward_noise)
        x = self.x + (cos(orientation) * dist)
        y = self.y + (sin(orientation) * dist)
        x %= world_size    # cyclic truncate
        y %= world_size
        
        # set particle
        res = robot()
        res.set(x, y, orientation)
        res.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        return res
    
    def Gaussian(self, mu, sigma, x):
        
        # calculates the probability of x for 1-dim Gaussian with mean mu and var. sigma
        return exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / sqrt(2.0 * pi * (sigma ** 2))
    
    
    def measurement_prob(self, measurement):
        
        # calculates how likely a measurement should be
        
        prob = 1.0;
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            prob *= self.Gaussian(dist, self.sense_noise, measurement[i])
        return prob
    
    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))


#myrobot = robot()
#myrobot.set_noise(5.0, 0.1, 5.0)
#myrobot.set(30.0, 50.0, pi/2)
#myrobot = myrobot.move(-pi/2, 15.0)
#print myrobot.sense()
#myrobot = myrobot.move(-pi/2, 10.0)
#print myrobot.sense()

myrobot = robot()
myrobot = myrobot.move(0.1, 5.0)
Z = myrobot.sense()

N = 1000
p = []
for i in range(N):
    x = robot()
    x.set_noise(0.05, 0.05, 5.0)
    p.append(x)

p2 = []
for i in range(N):
    p2.append(p[i].move(0.1, 5.0))
p = p2

w = []
for i in range(N):
    w.append(p[i].measurement_prob(Z))


#### DON'T MODIFY ANYTHING ABOVE HERE! ENTER CODE BELOW ####
# You should make sure that p3 contains a list with particles
# resampled according to their weights.
# Also, DO NOT MODIFY p.

p3 = []
# normalize weights
s=sum(w)
for i in range(len(w)):
    w[i]=w[i]/s

# cumulative weights
w_cumulative=[0]*1000
w_cumulative[0]=w[0]
for i in range(1,len(w)):
    w_cumulative[i]=w_cumulative[i-1]+w[i]

# select new particles
for i in range(0,len(w)):
    loop_flag=True
    while(loop_flag):
        rnd=random.random()
        for ix in range(0,len(w)):
            if(rnd<w_cumulative[ix]):
                p3.append(p[ix])
                loop_flag=False
                break



# Much Better explanation of resampling algorithm then in Udacity
https://robotics.stackexchange.com/questions/479/particle-filters-how-to-do-resampling/484

sample impoverishment. We can see why your approach suffers from it with a fairly simple example. Let's say you have 3 particles and their normalized weights are 0.1, 0.1, 0.8. Then multiplying by each weight by the 3 yields 0.3, 0.3, and 2.4. Then rounding yields 0, 0, 2. This means you would not pick the first two particles and the last one would be picked twice. Now you are down to two particles. I suspect this is what you have been seeing when you say "due to the roundoff errors, I end up having less particles."

An alternative selection method would be as follows.

    Normalize weights.
    Calculate an array of the cumulative sum of the weights.
    Randomly generate a number & determine which range in that cumulative weight array to which the number belongs.
    The index of that range would correspond to the particle that should be created.
    Repeat until you have the desired number of samples.

So, using the example above we would start with the normalized weights. We would then calculate the array [0.1, 0.2, 1]. From there we calculate 3 random numbers say 0.15, 0.38, and 0.54. This would have us pick the second particle once and the third particle twice. The point is that it gives the smaller particles a chance to propagate.

One thing to note is that while this method will deal with impoverishment it can lead to a suboptimal solutions. For instance, it may be that none of the particles really match your given location well (assuming you're using this for localization). The weights only tell you which particles match best, not the quality of the match. As such when you take additional readings and repeat the process you may find that all your particles group at a single location that is not the correct location. This is usually because there were no good particles to start. 